単純なアプリケーションではLLMを単独で使用するのは適切ですが、より複雑なアプリケーションでは、LLMを他のLLMや異なるコンポーネントと連鎖させる必要があります。> Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.LangChainは、コンポーネントを「連鎖」させるための2つの高水準フレームワークを提供しています。従来のアプローチでは`Chain`インターフェースを使用します。新しいアプローチでは、[LangChain Expression Language (LCEL)](/docs/expression_language/)を使用します。新しいアプリケーションを構築する際には、チェーンの構成にLCELを使用することを推奨します。しかし、依然として多くの有用な組み込み`Chain`をサポートしており、そのためここでは両方のフレームワークについて説明します。以下で触れるように、`Chain`自体もLCEL内で使用できるので、二つは相互排他的ではありません。> LangChain provides two high-level frameworks for "chaining" components. The legacy approach is to use the `Chain` interface. The updated approach is to use the [LangChain Expression Language (LCEL)](/docs/expression_language/). When building new applications we recommend using LCEL for chain composition. But there are a number of useful, built-in `Chain`'s that we continue to support, so we document both frameworks here. As we'll touch on below, `Chain`'s can also themselves be used in LCEL, so the two are not mutually exclusive.

## LCEL | LCELLCELの最も顕著な特徴は、構成のための直感的で読みやすい構文を提供していることです。しかし、それ以上に重要なのは、以下に挙げる機能に対して第一級のサポートを提供していることです：> The most visible part of LCEL is that it provides an intuitive and readable syntax for composition. But more importantly, it also provides first-class support for:* [ストリーミング](/docs/expression_language/interface#stream)、  > [streaming](/docs/expression_language/interface#stream),* [非同期呼び出し](/docs/expression_language/interface#async-stream)  > [async calls](/docs/expression_language/interface#async-stream),* [バッチ処理](/docs/expression_language/interface#batch)、  > [batching](/docs/expression_language/interface#batch),* [並列処理](/docs/expression_language/interface#parallelism)、  > [parallelization](/docs/expression_language/interface#parallelism),* 再試行回数  > retries,* [fallbacks](/docs/expression_language/how_to/fallbacks)（フォールバック）  > [fallbacks](/docs/expression_language/how_to/fallbacks),* トレーシング、  > tracing,* [さらに詳しく。](/docs/expression_language/why)  > [and more.](/docs/expression_language/why)簡単でありながら一般的な例として、プロンプト、モデル、そして出力パーサーを組み合わせた場合の様子を見てみましょう：> As a simple and common example, we can see what it's like to combine a prompt, model and output parser:

In [2]:
from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

model = ChatAnthropic()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a very knowledgeable historian who provides accurate and eloquent answers to historical questions.",
        ),
        ("human", "{question}"),
    ]
)
runnable = prompt | model | StrOutputParser()

In [3]:
for chunk in runnable.stream({"question": "How did Mansa Musa accumulate his wealth?"}):
    print(chunk, end="", flush=True)

 Mansa Musa was the emperor of the Mali Empire in West Africa during the 14th century. He accumulated immense wealth through several means:

- Gold mining - Mali contained very rich gold deposits, especially in the region of Bambuk. Gold mining and gold trade was a major source of wealth for the empire.

- Control of trade routes - Mali dominated the trans-Saharan trade routes connecting West Africa to North Africa and beyond. By taxing the goods that passed through its territory, Mali profited greatly.

- Tributary states - Many lands surrounding Mali paid tribute to the empire. This came in the form of gold, slaves, and other valuable resources.

- Agriculture - Mali also had extensive agricultural lands irrigated by the Niger River. Surplus food produced could be sold or traded. 

- Royal monopolies - The emperor claimed monopoly rights over the production and sale of certain goods like salt from the Taghaza mines. This added to his personal wealth.

- Inheritance - As an emperor, M

詳細は[LCELセクション](/docs/expression_language/)をご覧ください。> For more head to the [LCEL section](/docs/expression_language/).

## \[Legacy] `Chain` interface | \[レガシー] `Chain` インターフェース**Chain**は、「チェーン化」されたアプリケーションのためのレガシーインターフェイスです。Chainは、他のチェーンを含むコンポーネントへの一連の呼び出しとして、非常に一般的に定義されています。基本的なインターフェイスはシンプルです：> **Chain**'s are the legacy interface for "chained" applications. We define a Chain very generically as a sequence of calls to components, which can include other chains. The base interface is simple:```pythonclass Chain(BaseModel, ABC):    """Base interface that all chains should implement."""    memory: BaseMemory    callbacks: Callbacks    def __call__(        self,        inputs: Any,        return_only_outputs: bool = False,        callbacks: Callbacks = None,    ) -> Dict[str, Any]:        ...```上で作成したLCEL runnableを、組み込みの`LLMChain`を使用して再作成することができます：> We can recreate the LCEL runnable we made above using the built-in `LLMChain`:

In [4]:
from langchain.chains import LLMChain

chain = LLMChain(llm=model, prompt=prompt, output_parser=StrOutputParser())
chain.run(question="How did Mansa Musa accumulate his wealth?")

" Mansa Musa was the emperor of the Mali Empire in West Africa in the early 14th century. He accumulated his vast wealth through several means:\n\n- Gold mining - Mali contained very rich gold deposits, especially in the southern part of the empire. Gold mining and trade was a major source of wealth.\n\n- Control of trade routes - Mali dominated the trans-Saharan trade routes connecting West Africa to North Africa and beyond. By taxing and controlling this lucrative trade, Mansa Musa reaped great riches.\n\n- Tributes from conquered lands - The Mali Empire expanded significantly under Mansa Musa's rule. As new lands were conquered, they paid tribute to the mansa in the form of gold, salt, and slaves.\n\n- Inheritance - Mansa Musa inherited a wealthy empire from his predecessor. He continued to build the wealth of Mali through the factors above.\n\n- Sound fiscal management - Musa is considered to have managed the empire and its finances very effectively, including keeping taxes reasona

詳細については、以下をご覧ください：> For more specifics check out:* さまざまなチェーン機能のウォークスルーについては、[ハウツー](/docs/modules/chains/how_to/)をご覧ください。  > [How-to](/docs/modules/chains/how_to/) for walkthroughs of different chain features* [基礎](/docs/modules/chains/foundational/) コアとなるビルディングブロックチェーンについて学ぶために  > [Foundational](/docs/modules/chains/foundational/) to get acquainted with core building block chains* [ドキュメント](/docs/modules/chains/document/)を参照して、チェーンにドキュメントを組み込む方法を学んでください  > [Document](/docs/modules/chains/document/) to learn how to incorporate documents into chains